Library


In [511]:
import math
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import scipy.fftpack
import librosa
import soundfile as sf
from scipy.signal import resample_poly
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.tree import DecisionTreeClassifier, export_text
import os
from sklearn.linear_model import LogisticRegression

Main Path


In [512]:
# MAIN_PATH = r"C:\Users\Lulay\Documents\GitHub\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"
MAIN_PATH = r"D:\Kuliah\Matkul\Semester 4\DASAR KECERDASAN ARTIFICIAL (DKA)\[2] Tugas\[3] Tugas Besar\Dasar-Kecerdasan-Artificial_Tugas-Besar\Dataset"

Konstanta


In [513]:
SAMPLE_RATE = 48000
BATCH_SIZE = 32
SEGMENT_DURATION = int(1 * SAMPLE_RATE)
OVERLAP_DURATION = int(0.5 * SAMPLE_RATE)
RANDOM_STATE = 21
BUFFER_SIZE = 1000

Random Seed


In [514]:
RANDOM_SEED = 21

try:
    from sklearn.utils import check_random_state
    random_state = check_random_state(RANDOM_SEED)
except ImportError:
    pass

# Prapemrosesan


In [515]:
def load_file_audio(path):
    audio, sr = sf.read(path)
    return np.array(audio), sr

def prapemrosesan_downmixing(audio):
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    return audio.astype(np.float32)

def prapemrosesan_resampling(audio, sr):
    if sr == SAMPLE_RATE:
        return audio.copy(), SAMPLE_RATE
    
    ratio = SAMPLE_RATE / sr
    n_samples = int(np.round(len(audio) * ratio))
    
    x_old = np.linspace(0, 1, len(audio))
    x_new = np.linspace(0, 1, n_samples)
    return np.interp(x_new, x_old, audio), SAMPLE_RATE

def prapemrosesan_padding(audio):
    if np.mod(audio.shape[0], SEGMENT_DURATION) != 0:
        padding = SEGMENT_DURATION - (audio.shape[0] % SEGMENT_DURATION)
        audio = np.pad(audio, (0, padding))
    return audio

def prapemrosesan_splitting(audio):
    num_segments = int(np.floor((len(audio) - SEGMENT_DURATION) / OVERLAP_DURATION)) + 1
    segments = []

    for i in range(num_segments):
        start = int(i * OVERLAP_DURATION)
        end = int(start + SEGMENT_DURATION)
        segment = audio[start:end]
        if len(segment) < SEGMENT_DURATION:
            segment = np.pad(segment, (0, SEGMENT_DURATION - len(segment)), mode='constant')
        segments.append(segment)

    return np.array(segments)

# Ekstraksi Fitur


In [516]:
def get_rms(segment):
    return np.sqrt(np.mean(segment ** 2))

def get_zcr(segment):
    return np.sum(np.abs(np.diff(np.signbit(segment)))) / (len(segment) / SAMPLE_RATE)

def get_lms(segment):
    mel_spec = librosa.feature.melspectrogram(y=segment, sr=SAMPLE_RATE)
    return np.mean(mel_spec)

# Dataset


Raw


In [517]:
with open(f"{MAIN_PATH}/dataset_rmszcr.pkl", "rb") as f:
    dataset = pickle.load(f)

In [518]:
dataset[0]

('C:\\Users\\Lulay\\Documents\\GitHub\\Dasar-Kecerdasan-Artificial_Tugas-Besar\\Dataset\\xeno-canto\\19655.mp3',
 0.0,
 1.0,
 2,
 0.011059444,
 0.23255173703457446)

In [519]:
len(dataset)

865

Normalisasi, Split, dan Batching


In [520]:
X = [[data[4], data[5]] for data in dataset if data[3] != 2]
y = [int(data[3]) for data in dataset if data[3] != 2]

X_lr = [[data[4], data[5]] for data in dataset]
y_lr = [1 if int(data[3]) == 1 or int(data[3]) == 0 else 0 for data in dataset]

In [521]:
pd.Series(y).value_counts()

0    223
1    109
Name: count, dtype: int64

In [522]:
print(X[0],y[0])

[0.08369124, 0.24529379986702127] 0


In [523]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE, stratify=y_test)

In [524]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_lr, test_size=0.2, random_state=RANDOM_STATE, stratify=y_lr)

In [525]:
scl = StandardScaler()
X_train_scaled = scl.fit_transform(X_train)
X_test_scaled = scl.transform(X_test)
X_val_scaled = scl.transform(X_val)

In [526]:
X_train_scaled_lr = scl.fit_transform(X_train_lr)
X_test_scaled_lr = scl.transform(X_test_lr)

In [527]:
display(
    X_train_scaled.shape,
    X_test_scaled.shape,
    X_val_scaled.shape
)

(265, 2)

(33, 2)

(34, 2)

# LGBM


Inisialisasi


In [528]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)

training


In [529]:
lgbm_model.fit(X_train_scaled, y_train)

LGBMClassifier(random_state=21, verbose=-1)

# Decision Tree


Inisialisasi


In [530]:
dt_model = DecisionTreeClassifier(random_state=RANDOM_STATE)

training


In [531]:
dt_model.fit(X_train_scaled, y_train)

DecisionTreeClassifier(random_state=21)

## Logistic Regression

Inisialisasi

In [532]:
lr_model = LogisticRegression(random_state=RANDOM_STATE)

traning

In [533]:
lr_model.fit(X_train_scaled_lr, y_train_lr)

LogisticRegression(random_state=21)

# CNN


Inisialisasi


In [534]:
# class CNN(nn.Module):
#     def __init__(self, num_classes=10):
#         super(CNN, self).__init__()
#         test_input = np.random.rand(SAMPLE_RATE * SEGMENT_DURATION)
#         lms = get_lms(test_input)
#         in_channels = 1
#         self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(128)
#         self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(256)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.gap = nn.AdaptiveAvgPool2d(1)
#         self.fc1 = nn.Linear(256, 256)
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(256, num_classes)
#     def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))
#         x = self.pool(F.relu(self.bn3(self.conv3(x))))
#         x = self.gap(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

training


# Conformer


Inisialisasi


In [535]:
# config = ConformerConfig(
#     hidden_size=144,
#     num_attention_heads=4,
#     intermediate_size=576,
#     conv_kernel_size=32,
#     num_hidden_layers=8,
#     input_feat_per_channel=n_mels,
#     input_channels=1,
#     max_position_embeddings=time_steps
# )

# conformer_model = TFConformerModel(config)

# input_layer = tf.keras.layers.Input(shape=(n_mels, time_steps))
# expand_dim = tf.keras.layers.Reshape((1, n_mels, time_steps))(input_layer)
# conv_proj = tf.keras.layers.Conv2D(
#     filters=config.hidden_size,
#     kernel_size=(3, 3),
#     padding='same',
#     activation='relu'
# )(expand_dim)
# squeeze_dim = tf.keras.layers.Reshape((n_mels, time_steps, config.hidden_size))(conv_proj)
# conformer_output = conformer_model(squeeze_dim).last_hidden_state
# gap = tf.keras.layers.GlobalAveragePooling1D()(conformer_output)
# output = tf.keras.layers.Dense(num_classes, activation='softmax')(gap)

# conformer_model = tf.keras.Model(inputs=input_layer, outputs=output)

training


# Model Lain


inisialisasi


In [536]:
# models = {
#     "rf": RandomForestClassifier(random_state=RANDOM_STATE),
#     "xg": XGBClassifier(random_state=RANDOM_STATE),
#     "cat": CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
#     "gb": GradientBoostingClassifier(random_state=RANDOM_STATE),
#     "knn": KNeighborsClassifier(),
#     "dt": DecisionTreeClassifier(random_state=RANDOM_STATE),
#     "svm": SVC(random_state=RANDOM_STATE)
# }

training


In [537]:
# results = {}

# for model_name, model in models.items():
#     model.fit(X_train_scaled, y_train)
#     y_pred = model.predict(X_test_scaled)
    
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
#     cm = confusion_matrix(y_test, y_pred)
    
#     results[model_name] = {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1_score': f1,
#         'confusion_matrix': cm.tolist()
#     }

# Evaluasi


LGBM


In [538]:
y_pred = lgbm_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = lgbm_model.predict(X_test_scaled)

print("\n")
print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Validasi
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
 [[23  0]
 [ 0 11]]


Test
Accuracy: 0.9393939393939394
Precision: 0.9090909090909091
Recall: 0.9090909090909091
F1 Score: 0.9090909090909091
Confusion Matrix:
 [[21  1]
 [ 1 10]]


CNN


Conformer


Descision Tree


In [539]:
y_pred = dt_model.predict(X_val_scaled)

print("Validasi")
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_val, y_pred)}")

y_pred = dt_model.predict(X_test_scaled)

print("\n")
print("Test")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

Validasi
Accuracy: 0.9705882352941176
Precision: 1.0
Recall: 0.9090909090909091
F1 Score: 0.9523809523809523
Confusion Matrix:
 [[23  0]
 [ 1 10]]


Test
Accuracy: 0.9090909090909091
Precision: 0.8333333333333334
Recall: 0.9090909090909091
F1 Score: 0.8695652173913043
Confusion Matrix:
 [[20  2]
 [ 1 10]]


In [540]:
feature_names = ['RMS', 'ZCR']
tree_rules = export_text(dt_model, feature_names=feature_names)
print("Extracted Decision Rules:")
print(tree_rules)

Extracted Decision Rules:
|--- RMS <= 0.04
|   |--- ZCR <= -1.46
|   |   |--- class: 1
|   |--- ZCR >  -1.46
|   |   |--- RMS <= -0.21
|   |   |   |--- ZCR <= 2.15
|   |   |   |   |--- RMS <= -0.38
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- RMS >  -0.38
|   |   |   |   |   |--- RMS <= -0.38
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- RMS >  -0.38
|   |   |   |   |   |   |--- class: 0
|   |   |   |--- ZCR >  2.15
|   |   |   |   |--- class: 1
|   |   |--- RMS >  -0.21
|   |   |   |--- ZCR <= -0.09
|   |   |   |   |--- ZCR <= -0.56
|   |   |   |   |   |--- ZCR <= -0.83
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- ZCR >  -0.83
|   |   |   |   |   |   |--- ZCR <= -0.64
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- ZCR >  -0.64
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- ZCR >  -0.56
|   |   |   |   |   |--- class: 1
|   |   |   |--- ZCR >  -0.09
|   |   |   |   |--- RMS <= -0.21
|   |   |   |   |  

Model Lain


In [541]:
# for model_name, model in models.items():
#     print(f"{model_name} Model:")
#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1 Score: {f1:.4f}")
#     print(f"Confusion Matrix:\n{cm}")
#     print("-" * 40)

Logistic Regression

In [542]:
y_pred = lr_model.predict(X_test_scaled_lr)
print("Test")
print(f"Accuracy: {accuracy_score(y_test_lr, y_pred)}")
print(f"Precision: {precision_score(y_test_lr, y_pred)}")
print(f"Recall: {recall_score(y_test_lr, y_pred)}")
print(f"F1 Score: {f1_score(y_test_lr, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test_lr, y_pred)}")

Test
Accuracy: 0.8265895953757225
Precision: 0.8214285714285714
Recall: 0.696969696969697
F1 Score: 0.7540983606557377
Confusion Matrix:
 [[97 10]
 [20 46]]


# Visualisasi Hasil


In [543]:
def load_and_plot(path, list_result, final_time):
    file_name = os.path.basename(path)
    audio, sr = librosa.load(path, sr=SAMPLE_RATE)
    time_axis = np.linspace(0, len(audio) / sr, len(audio))

    plt.figure(figsize=(20, 4))
    plt.plot(time_axis, audio, label="Amplitude")
    plt.title(f"Audio waveform with loud segments marked - {file_name}") 
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    for i in enumerate(list_result):
        start_time = i[1][0]
        end_time = i[1][1]
        plt.axvspan(start_time, end_time, color='red', alpha=0.3)

    plt.xticks(np.arange(0, final_time, 0.5))
    plt.grid(which='both', alpha=0.5)
    plt.xlim(0, final_time)
    plt.ylim(-1, 1)
    plt.tight_layout()
    plt.legend()
    plt.show()

### LGBM


In [544]:
def single_inference(path):
    audio, sr = load_file_audio(path)
    audio = prapemrosesan_downmixing(audio)
    audio, sr = prapemrosesan_resampling(audio, sr)
    audio = prapemrosesan_padding(audio)
    segments = prapemrosesan_splitting(audio)
    
    list_result = []
    for index, segment in enumerate(segments):
        rms = get_rms(segment)
        zcr = get_zcr(segment)
        X = np.array([rms, zcr])
        X = scl.transform(X.reshape(1, -1))

        y_pred = lgbm_model.predict(X)
        if y_pred == 0:
            list_result.append(([index*0.5, index*0.5 + 1]))
    return list_result

In [545]:
# for file in os.listdir(f"{MAIN_PATH}/noise-audio-data"):
#     list_result = single_inference(f"{MAIN_PATH}/noise-audio-data/{file}")
#     print(list_result)
#     load_and_plot(f"{MAIN_PATH}/noise-audio-data/{file}", list_result, 5)  

In [546]:
# for file in os.listdir(f"{MAIN_PATH}/xeno-canto"):
#     list_result = single_inference(f"{MAIN_PATH}/xeno-canto/{file}")
#     print(list_result)
#     load_and_plot(f"{MAIN_PATH}/xeno-canto/{file}", list_result, 25)  